# Guided Exploration - Infer MITRE technique from Threat Intel Data

__Notebook Version:__ 1.0 <br>
__Notebook Author:__ Vani Asawa<br>


__Python Version:__ >=Python 3.8<br>
__Platforms Supported:__  Azure Machine Learning Notebooks<br>

__Data Source Required:__ None<br>

__GPU Compute Required:__ No<br>
__GPU Compute Recommended:__ Yes<br>

__Requirements Path:__ ```Guided Exploration - Infer MITRE technique from Threat Intel Data/requirements.txt```<br>
__Essential Packages:__ 
- ipywidgets==7.5.1
- transformers==4.5.1
- torch==1.10.2
- msticpy==2.1.2
- nltk==3.6.2
- iocextract==1.13.1
- shap==0.41.0

## Description
**Cyber Threat Intelligence** (CTI) provides a framework for threat analysts to document the operations of a threat actor group and record the findings of their investigations of specific cyber attack incidents.

With the increasing number and sophistication of attacks occuring across organization's workspace CTI allows organisations to develop a more robust and proactive security posture better detect threat vulnerabilities in their infrastructre and adopt security solutions and policies that allow them to better protect their environment. For example **Indicators of Compromise (IoC)** represent network artifacts of a cyber intrusion and are widely used in intrusion detection systems and antivirus softwares to detect future attacks.

**Threat Intel Data** is another form of CTI which comprises of rich unstructured textual data describing the tools techniques and procedures used by threat actor groups in a cyber operation. Historically TI data is made available to the security community in the form of blog posts reports and white papers. With the increasing numebr of cyber attacks it is not scalable to manually process this growing corpus of TI data to understand the motivations capabilities and TTPs associated with an actor group. Additionally TI data does not facilitate easy extraction of IoCs which if documented in the report can result in the loss of known indicators in the threat intelligence corpus. This opens up several avenues for **Machine Learning** more particularly **Natural Language Processing** (NLP) to identify TTPs and IoCs from this data.

The **MITRE ATT&CK** framework is an openly-sourced knowledge base of TTPs used by adversaries across enterprise and mobile applications. MITRE TTPs allow people and organizations to proactively identify vulnerabilites in their system based on the behaviors methods and patterns of activity used by an actor group in different stages of a cyber operation.

#################################

In this notebook we use NLP to
1. *Detect MITRE TTPs* &
2. *Extract IoCs*

from unstructured English text-based Threat Intel data. We also provide some explainability into the TTP predictions made by our NLP model by identifying specific words or phrases in the input TI data that contribute to the prediction.

#################################

## Prerequisites
**Please do not run the notebook cells all at once**. The cells need to be run sequentially and successfully executed before proceeding with the remainder of the notebook.

## Table of Contents

1. Imports
2. Configure Input Data and Model Parameters
3. Get Model Artifacts
4. Process TI Data
5. Inference
6. Explainability

## Installations

Please download the packages in ```Guided Exploration - Infer MITRE technique from Threat Intel Data/requirements.txt``` in your virtual environment before running the rest of the cells in the notebook.

In [1]:
import os
import sys
sys.path.append(os.getcwd())

############### REQUIREMENTS.TXT NEEDS TO BE UPDATED ################
# requirements_path = os.path.join(os.getcwd() 'requirements.txt')
# os.system(f'pip install -r {requirements_path}')

##  Imports

The modules used to run this notebook can be found under ```Guided Exploration - Infer MITRE technique from Threat Intel Data/utils/*```

In [ ]:
import torch
import utils
from utils import (
    configs as config_utils,
    storage as storage_utils,
    inference as inference_utils,
    process as process_utils,
)

## Configure Input Data and Model Parameters,
The notebook requires the following parameters from the user:
1. ***Threat Intel Data***: Unstructured, English text data that the user would like to process through the NLP model. If you are inputting multiple text reports in the widget, please input the reports separated by an empty line. Do not include any commas, punctuations, or brackets before and after the reports. <br>
- For example: Here, we are processing three different threat reports, which are separated by an empty line. The length of each report can be more than one sentence. In this example, for the purposes of succinct documentation, the length of each report is 1 sentence.

    ```
    Like many threat groups, TG-3390 conducts strategic web compromises (SWCs), also known as watering hole attacks, on websites associated with the target organization's vertical or demographic to increase the likelihood of finding victims with relevant information.

    Threat groups use strategic web compromises (SWCs), also known as watering hole attacks, to target a wide array of potential victims.

    A build tool is likely being used by these attackers that allows the operator to configure details such as C2 addresses, C2 encryption keys, and a campaign code.
    ```
       
2. ***Select NLP Model***: We have trained four variations of GPT-2 transformer models using publicly-available threat intel datasets that map TI data to MITRE TTPs. 
- *distilgpt2* models are 40% lower in storage size than the *gpt2* models <br>

- *distilgpt2-1024* and *gpt2-1024* models process more word tokens in a single threat input statement than the *distilgpt2-512* and *distilgpt2-1024* models, which can be particularly useful if your threat intel data is long. <br>

- Default model: **distilgpt2_512** <br><br>

3. ***Minimum Score Threshold***: The TTP predictions for a sample TI input data have an associated confidence score from the NLP model, ranging from 0 (less confident) to 1 (most confident). Filter the results to predictions with confidence >= threshold configured by the user. <br>

- Default threshold: **0.7** <br> <br>
       
4. ***Chunk Threat Intel Data?***: 
- One of the limitations of the transformer models is that they can only process inputs upto a certain length, after which the rest of the report is discarded. 
- As a result, the model will lose out on potentially important information about the actor's TTPs, described in the latter parts of the report. 
- If a single threat report in your input is longer than 3 sentences, we recommend **chunking** - The model will process the sentences in your input data in batches of 3 sentences, hence assigning a TTP prediction for each chunk of data, and processing the entire report. <br>

- Default value: **Yes** <br><br>

5. ***Extract Indicators of Compromise (IoCs)***: Extract IoCs from the input TI data. <br>

- Default value: **Yes** <br><br>

6. ***Get NLP Model Explainability***: Obtain further insights into which words and phrases in your input data contributed to the TTP prediction. <br>

- Default value: **Yes** <br><br>

In [ ]:
all_config_widgets = config_utils.configure_model_parameters()
for k in all_config_widgets.keys():
       display(all_config_widgets[k])

In [ ]:
set_configs = {
    k: v.value for k, v in all_config_widgets.items()
}

configs = config_utils.format_user_configuration(set_configs, verbose=True)

## Download Model Artifacts

In [ ]:
! bash ./models.sh {configs['model']}

In [ ]:
assets = storage_utils.AssetStorage(
    configs['model']
)

model, tokenizer, labels = assets.model, assets.tokenizer, assets.labels
device = assets.device

In [ ]:
inference_model = inference_utils.InferenceClassificationPipeline(
    model = model,
    tokenizer = tokenizer,
    device = device.type
)

## Process TI Data

In [ ]:
processed_data_object = process_utils.ProcessData(
    configs = configs
)

processed_data_object.go()

## Inference

In [ ]:
outputs = inference_model.go(processed_data_object.processed_data)

In [ ]:
inference_df = inference_utils.format_predictions(
    configs = configs,
    processed_data_object = processed_data_object,
    labels = labels,
    outputs = outputs,
    classifier = inference_model.classifier_max_scores
)

In [ ]:
print(f'Shape of DF: {inference_df.shape}')
       
print('Sample Result: ')
if inference_df.empty:
    print('Empty Dataframe')
else:
    display(inference_df.head(1))

## Explainability - Visualisation

In [ ]:
row_index = 2
       
inference_utils = inference_utils.process_shap_explainability_for_row(
    inference_df, row_index
)